<a href="https://colab.research.google.com/github/Sadickachuli/Automated-Grant-Impact-Analysis/blob/main/autoGIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf nltk spacy transformers streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.3 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded = files.upload()


Saving USAID.pdf to USAID.pdf


In [ ]:
import fitz

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

pdf_text = extract_text_from_pdf("USAID.pdf")
print(pdf_text[:1000])  # Printing first 1000 characters


USAID GLOBAL HEALTH SUPPLY CHAIN – 
PROCUREMENT AND SUPPLY 
MANAGEMENT PROJECT 
QUARTERLY REPORT 
YEAR 1 QUARTER 1, JANUARY 11 TO MARCH 31, 2016 
April 30, 2016 
DISCLAIMER 
The authors’ views expressed in this publication do not necessarily reflect the views of the United 
States Agency for International Development or of the United States government. 

This publication was produced for review by the United States Agency for International 
Development. It was prepared by Chemonics International Inc. under Global Health Supply 
Chain - Procurement and Supply Management (GHSC-PSM) Contract No. AID-OAA-I-15-
00004; Task Order 01 Contract No. AID-OAA-TO-15-00007; Task Order 02 Contract No. 
AID-OAA-TO-15-00009; and Task Order 03 Contract No. AID-OAA-TO-15-00010. 
Recommended citation: USAID Global Health Supply Chain – Procurement and Supply 
Management Single Award IDIQ. 2016. USAID GHSC-PSM Quarterly Report Year 1 Quarter 1. 
Washington, D.C.: Chemonics International Inc. 
Chemonics Con

In [ ]:
import nltk
import spacy
import string
from nltk.corpus import stopwords

# Downloading necessary NLP resources
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    doc = nlp(text.lower())  # Convert to lowercase
    tokens = [token.lemma_ for token in doc if token.is_alpha and token.text not in stop_words]  # For Lemmatization
    return " ".join(tokens)

cleaned_text = preprocess_text(pdf_text)
print(cleaned_text[:500])  # Print first 500 characters


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


usaid global health supply chain procurement supply management project quarterly report year quarter january march april disclaimer author view express publication necessarily reflect view united states agency international development united states government publication produce review united states agency international development prepare chemonics international inc global health supply chain procurement supply management ghsc psm contract aid oaa task order contract aid oaa task order contrac


In [ ]:
from collections import Counter

def extract_keywords(text, top_n=10):
    doc = nlp(text)
    keywords = [token.text for token in doc if token.pos_ in ["NOUN", "PROPN"]]  # Extract nouns and proper nouns
    keyword_freq = Counter(keywords).most_common(top_n)
    return keyword_freq

key_themes = extract_keywords(cleaned_text, top_n=10)
print("Key Themes:", key_themes)


Key Themes: [('psm', 190), ('ghsc', 187), ('plan', 122), ('transition', 118), ('team', 99), ('country', 85), ('supply', 83), ('commodity', 82), ('quarter', 75), ('project', 73)]


In [ ]:
def extract_named_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

impact_areas = extract_named_entities(cleaned_text)
print("Impact Areas:", impact_areas[:10])  # Print first 10 entities


Impact Areas: [('quarterly', 'DATE'), ('year quarter january march april', 'DATE'), ('united states agency', 'ORG'), ('united states', 'GPE'), ('chemonics international inc global health supply chain', 'ORG'), ('quarterly', 'DATE'), ('year quarter', 'DATE'), ('washington chemonics international inc chemonic', 'ORG'), ('anthony savelli', 'PERSON'), ('arlington va p', 'FAC')]


In [ ]:
from transformers import pipeline

# Load sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis")

# Run sentiment analysis on the extracted text
sentiment_result = sentiment_pipeline(cleaned_text[:512])  # Limit input due to model constraints
print("Sentiment Analysis Result:", sentiment_result)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Sentiment Analysis Result: [{'label': 'NEGATIVE', 'score': 0.8508706092834473}]
